In [1]:
!pip install Chroma

  Preparing metadata (setup.py) ... done
  Created wheel for Chroma: filename=Chroma-0.2.0-py3-none-any.whl size=7095 sha256=c32c1dfaafc7dc4ff39e0aca39c63dbc59f8c7bf226df37c10b086a8abeb812a
  Stored in directory: /root/.cache/pip/wheels/58/74/75/a6ab7999ae473ecbe819bc5cae9ccb902429dd6c60795f5112
Successfully built Chroma


In [30]:
!pip install OpenAI

In [32]:
from openai import OpenAI

In [2]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


In [3]:
!pip -q install langchain openai tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip
!unzip -q new_articles.zip -d new_articles

In [5]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [7]:
import os
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [8]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

In [10]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader


In [11]:
loader=DirectoryLoader('/content/new_articles',glob="./*.txt",loader_cls=TextLoader)

In [12]:
documents=loader.load()

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [15]:
textsplitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
texts=textsplitter.split_documents(documents)

In [16]:
len(texts)

233

In [17]:
persist_directory='db'
embedding=OpenAIEmbeddings()
vectordb=Chroma.from_documents(documents=texts,
                               embedding=embedding,
                               persist_directory=persist_directory)

In [18]:
vectordb.persist()
vectordb=None

In [19]:
vectordb=Chroma(persist_directory=persist_directory,
                embedding_function=embedding)

In [21]:
retriever=vectordb.as_retriever()

In [22]:
docs=retriever.get_relevant_documents("How much money did Microsoft raise?")

In [23]:
docs

[Document(page_content='April 28, 2023\n\nVC firms including Sequoia Capital, Andreessen Horowitz, Thrive and K2 Global are picking up new shares, according to documents seen by TechCrunch. A source tells us Founders Fund is also investing. Altogether the VCs have put in just over $300 million at a valuation of $27 billion to $29 billion. This is separate to a big investment from Microsoft announced earlier this year, a person familiar with the development told TechCrunch, which closed in January. The size of Microsoft’s investment is believed to be around $10 billion, a figure we confirmed with our source.\n\nApril 25, 2023\n\nCalled ChatGPT Business, OpenAI describes the forthcoming offering as “for professionals who need more control over their data as well as enterprises seeking to manage their end users.”', metadata={'source': '/content/new_articles/05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt'}),
 Document(page_content='The amount that Google investe

In [24]:
len(docs)

4

In [25]:
retriever=vectordb.as_retriever(search_kwargs={"k":2})

In [26]:
retriever.search_type


'similarity'

In [38]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [39]:
qa_chain=RetrievalQA.from_chain_type(llm=OpenAI(),
                                     chain_type="stuff",
                                     retriever=retriever,
                                     return_source_documents=True)

In [40]:
query="How much money did Microsoft raise?"
llm_response=qa_chain(query)

In [41]:
llm_response

{'query': 'How much money did Microsoft raise?',
 'result': '\n$10 billion',
 'source_documents': [Document(page_content='April 28, 2023\n\nVC firms including Sequoia Capital, Andreessen Horowitz, Thrive and K2 Global are picking up new shares, according to documents seen by TechCrunch. A source tells us Founders Fund is also investing. Altogether the VCs have put in just over $300 million at a valuation of $27 billion to $29 billion. This is separate to a big investment from Microsoft announced earlier this year, a person familiar with the development told TechCrunch, which closed in January. The size of Microsoft’s investment is believed to be around $10 billion, a figure we confirmed with our source.\n\nApril 25, 2023\n\nCalled ChatGPT Business, OpenAI describes the forthcoming offering as “for professionals who need more control over their data as well as enterprises seeking to manage their end users.”', metadata={'source': '/content/new_articles/05-03-chatgpt-everything-you-need-t

In [42]:
def process_llm_response(llm_response):
  print(llm_response['result'])
  print('\n\nSources:')

In [43]:
query="What is spacex?"
llm_response=qa_chain(query)
process_llm_response(llm_response)

 SpaceX is a private aerospace company founded by entrepreneur Elon Musk. It designs, manufactures, and launches advanced rockets and spacecraft with the goal of enabling the colonization of Mars and further exploration of space. SpaceX's most well-known project is the Falcon 9 rocket and Dragon spacecraft, which have been used to resupply the International Space Station and launch satellites into orbit. The company is also developing a super-heavy launch system called Starship, which aims to drastically lower the cost of launching mass into space. 


Sources:
